In [1]:
import pandas as pd
import numpy as np
import time 

user_file_name = "data_line_balancing.xlsx"#input("Enter your excel file name with extension: ")
user_sheet_name = "GroupbyCardTypeLST"#input("Enter your excel's sheet name:  ")
st = time.time()
df_data = pd.read_excel(user_file_name,sheet_name=user_sheet_name)
num_operator = pd.read_excel("data_line_balancing.xlsx",sheet_name="NumOperator")
lines = 9
cards = 25
orders = len(df_data)
operators = 36
cards_pallets = [17,18,19,20,22]
setup = 900/(24*60*60)
T = np.zeros((orders,lines),dtype=list)
L = np.zeros((orders,lines),dtype=int)
C = np.zeros((orders,lines),dtype=float)
O = np.zeros((operators,lines),dtype=list)
index_data = df_data.index
index_operator = num_operator.index

val = []
for i in index_data:
    for j in index_operator:
        if df_data["Card Type"].iloc[i] == num_operator["Card Type"].iloc[j]:
            value = num_operator["Needed Operator"].iloc[j]
            val.append(value)

val = pd.DataFrame(val)
df_data["Needed Operator"] = val

value = []
for i in index_data:
    if df_data["Card Type"].iloc[i] in cards_pallets :
        a = "Pallets"
        value.append(a)
    else:
        b = "Non_Pallets"
        value.append(b)
df_data["Process Type"] = value

data_pallets = df_data[df_data["Process Type"]=="Pallets"]
data_pallets = data_pallets.reset_index()
data_pallets = data_pallets.drop(columns={"index"})
data_pallets = data_pallets.sort_values(["Card Type","Latest Start Time"],ascending=[True,True])
data_pallets = data_pallets.reset_index()
data_pallets = data_pallets.drop(columns={"index"})

data_non_pallets = df_data[df_data["Process Type"]=="Non_Pallets"]
data_non_pallets = data_non_pallets.reset_index()
data_non_pallets = data_non_pallets.drop(columns={"index"})
data_non_pallets = data_non_pallets.sort_values(["Card Type","Latest Start Time"],ascending=[True,True])
data_non_pallets = data_non_pallets.reset_index()
data_non_pallets = data_non_pallets.drop(columns={"index"})

index_pallets = data_pallets.index
index_non_pallets = data_non_pallets.index

def assign_to_non_pallets(order_index):
    for i in range(3,9):
        line_process_time =(data_non_pallets["Process Time"].iloc[order_index]/6)/(24*60*60)
        if order_index == 0:
            C[order_index][i] = line_process_time
        elif order_index != 0 and data_non_pallets["Card Type"].iloc[order_index] == data_non_pallets["Card Type"].iloc[order_index-1]:
            C[order_index][i] = line_process_time + C[order_index-1][i]
        else:
            C[order_index][i] = line_process_time + C[order_index-1][i] + setup

for i in index_non_pallets:
    assign_to_non_pallets(i)


def assign_non_pallets_order_no(order_index):
    for i in range(3,9):
        order_no = data_non_pallets["Orders"].iloc[order_index]
        T[order_index][i] = order_no
        
for i in index_non_pallets:
    assign_non_pallets_order_no(i)

def assign_to_pallets(order_index):
    for i in range(0,3):
        line_process_time = (data_pallets["Process Time"].iloc[order_index]/3)/(24*60*60)
        if order_index == 0:
            C[order_index][i] = line_process_time
        elif order_index != 0 and data_pallets["Card Type"].iloc[order_index] == data_pallets["Card Type"].iloc[order_index-1]:
            C[order_index][i] = line_process_time + C[order_index-1][i]
        else:
            C[order_index][i] = line_process_time + C[order_index-1][i] + setup

for i in index_pallets:
    assign_to_pallets(i)

def assign_pallets_order_no(order_index):
    for i in range(0,3):
        order_no = data_pallets["Orders"].iloc[order_index]
        T[order_index][i] = order_no
        
for i in index_pallets:
    assign_pallets_order_no(i)

c = pd.DataFrame(C)

val = []
for i in index_data:
    for j in index_operator:
        if df_data["Card Type"].iloc[i] == num_operator["Card Type"].iloc[j]:
            value = num_operator["Needed Operator"].iloc[j]
            val.append(value)

val = pd.DataFrame(val)
df_data["Needed Operator"] = val



opr_val = []
for j in range(len(T)):
    for i in range(9):
        for k in range(len(T)):
            if T[j][i]== df_data["Orders"].iloc[k]:
                result = df_data["Needed Operator"].iloc[k]
                L[j][i] = result
L = pd.DataFrame(L)


T = pd.DataFrame(T)
T = T.rename(columns={0:"1. Line",1:"2. Line",2:"3. Line",3:"4. Line",4:"5. Line",5:"6. Line",6:"7. Line",7:"8. Line",8:"9. Line"})

num_operator_index = L.index

max_num_pallets = []
max_num_non_Pallets = []
for i in num_operator_index:
    max_operator = L[0].iloc[i]
    max_num_pallets.append(max_operator)
for i in num_operator_index:
    max_operator = L[3].iloc[i]
    max_num_non_Pallets.append(max_operator)
max_num_pallets = max(max_num_pallets)
max_num_non_Pallets = max(max_num_non_Pallets)

def operator_assign_to_pallet_lines(operator_index):
    iteration = 0
    for j in range(3):
        for k in range(iteration,max_num_pallets+iteration):
            O[k][j] = 1
        iteration += max_num_pallets
operator_assign_to_pallet_lines(max_num_pallets)

def operator_assign_to_non_pallet_lines(operator_index):
    iteration = 0
    for j in range(3,9):
        for k in range(iteration+max_num_pallets*3,max_num_non_Pallets+iteration+max_num_pallets*3):
            O[k][j] = 1
        iteration += max_num_non_Pallets
operator_assign_to_non_pallet_lines(max_num_non_Pallets)

name = []
for i in range(1,operators+1):
    write = f"Operator{i}"
    name.append(write)

O = pd.DataFrame(O)
O = O.rename(columns={0:"1. Line",1:"2. Line",2:"3. Line",3:"4. Line",4:"5. Line",5:"6. Line",6:"7. Line",7:"8. Line",8:"9. Line"})
O["Operator No"] = name
last_col = O.pop("Operator No")
O.insert(0,"Operator No",last_col)

value_B = []
for i in range(lines):
    sum_lines = max(c[i])
    value_B.append(sum_lines)
max(value_B)
c = c.rename(columns={0:"1. Line",1:"2. Line",2:"3. Line",3:"4. Line",4:"5. Line",5:"6. Line",6:"7. Line",7:"8. Line",8:"9. Line"})
print(f"maximum time: {max(value_B)}")

#If you want to write the solutions in a different excel file you use the code in the commands:

#solution_name = input("Process completed. Enter your new excel file name to write down your solution with extension: ")

#writer = pd.ExcelWriter(solution_name, engine="xlsxwriter")
#T.head(len(index_data)).to_excel(writer, sheet_name="Orders in Lines")
#c.head(len(index_data)).to_excel(writer, sheet_name="Completion Time of Orders")
#O.head(len(O)).to_excel(writer, sheet_name="Operator assignment to Lines")
#writer.save()

et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

maximum time: 11.37695674296982
Execution time: 0.6807351112365723 seconds


In [19]:
c

,1. Line,2. Line,3. Line,4. Line,5. Line,6. Line,7. Line,8. Line,9. Line
0,0.113481,0.113481,0.113481,0.111506,0.111506,0.111506,0.111506,0.111506,0.111506
1,0.527216,0.527216,0.527216,0.151417,0.151417,0.151417,0.151417,0.151417,0.151417
2,2.891414,2.891414,2.891414,0.222262,0.222262,0.222262,0.222262,0.222262,0.222262
3,3.482463,3.482463,3.482463,1.077441,1.077441,1.077441,1.077441,1.077441,1.077441
4,3.659778,3.659778,3.659778,1.302711,1.302711,1.302711,1.302711,1.302711,1.302711
5,4.250827,4.250827,4.250827,1.626400,1.626400,1.626400,1.626400,1.626400,1.626400
6,5.432926,5.432926,5.432926,1.703518,1.703518,1.703518,1.703518,1.703518,1.703518
7,5.661904,5.661904,5.661904,1.736869,1.736869,1.736869,1.736869,1.736869,1.736869
8,5.747515,5.747515,5.747515,1.757677,1.757677,1.757677,1.757677,1.757677,1.757677
9,6.198684,6.198684,6.198684,1.996392,1.996392,1.996392,1.996392,1.996392,1.996392


In [22]:
T

,1. Line,2. Line,3. Line,4. Line,5. Line,6. Line,7. Line,8. Line,9. Line
0,order37,order37,order37,order1,order1,order1,order1,order1,order1
1,order38,order38,order38,order2,order2,order2,order2,order2,order2
2,order39,order39,order39,order3,order3,order3,order3,order3,order3
3,order40,order40,order40,order4,order4,order4,order4,order4,order4
4,order41,order41,order41,order5,order5,order5,order5,order5,order5
5,order42,order42,order42,order6,order6,order6,order6,order6,order6
6,order43,order43,order43,order7,order7,order7,order7,order7,order7
7,order44,order44,order44,order8,order8,order8,order8,order8,order8
8,order45,order45,order45,order9,order9,order9,order9,order9,order9
9,order46,order46,order46,order10,order10,order10,order10,order10,order10


In [23]:
O

,Operator No,1. Line,2. Line,3. Line,4. Line,5. Line,6. Line,7. Line,8. Line,9. Line
0,Operator1,1,0,0,0,0,0,0,0,0
1,Operator2,1,0,0,0,0,0,0,0,0
2,Operator3,0,1,0,0,0,0,0,0,0
3,Operator4,0,1,0,0,0,0,0,0,0
4,Operator5,0,0,1,0,0,0,0,0,0
5,Operator6,0,0,1,0,0,0,0,0,0
6,Operator7,0,0,0,1,0,0,0,0,0
7,Operator8,0,0,0,1,0,0,0,0,0
8,Operator9,0,0,0,1,0,0,0,0,0
9,Operator10,0,0,0,1,0,0,0,0,0
